In [11]:
# 라이브러리 불러오기기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import json
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pydantic import BaseModel
from typing import List, Optional

In [12]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'data')

In [13]:
data_file_path = os.path.join(data_directory, 'filter_df.pickle')
df = pd.read_pickle(data_file_path)

In [14]:
df = df[~df['title'].str.contains('임대|야놀자|입장권|상품권|포인트|야놀|주차권|쿠폰|구매|비행기|종일권|자유이용권', na=False)]
df.reset_index(drop=True, inplace=True)

In [15]:
load_dotenv()

openai.api_key = os.getenv('OPEN_AI_KEY')

In [16]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

In [17]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

# Pydantic을 사용해 응답 데이터를 구조화할 모델 정의.
class Fields(BaseModel):
    expiration_stdate: Optional[str]
    expiration_endate: Optional[str]
    room_type: Optional[str]
    head_count: Optional[int]
    shipping_fee: Optional[int]
    transaction_method: Optional[str]
    market_price: Optional[str]
    options: Optional[str]
    parking: Optional[bool]
    check_in_time: Optional[str]
    check_out_time: Optional[str]
    stay_type: Optional[str]
    foreign_yn: Optional[str]
    event: Optional[str]

# 함수를 정의하여 데이터를 요청하고 응답 처리.
def extract_fields(description):
    prompt = {
        "role": "system", 
        "content": """
        너는 데이터 필드를 추출해주는 도우미야. 데이터를 받아서 필요한 필드를 14개로 제한해 JSON 형식으로 응답할거야. 
        설명에서 필요한 14개의 필드를 정확히 추출하고 나머지 필드들은 무시해줘. 필드는 오직 14개만 있어야 하고, 추가적인 세부 필드를 포함해서는 안 돼.
        NaN이나 정보가 없으면 해당 필드를 null로 설정해줘.
        필드는 아래 14개:
        1. expiration_stdate (해당 상품의 사용기한 중 시작 날짜 YYYY-MM-DD 형식)
        2. expiration_endate (해당 상품의 사용기한 중 마감 날짜 YYYY-MM-DD 형식)
        3. room_type (객실 유형)
        4. head_count (사용 인원수, int로만 알려줘)
        5. shipping_fee (배송비, 없으면 0)
        6. transaction_method (직거래/택배거래 여부)
        7. market_price (시중 가격, int로만 알려줘)
        8. options (호텔 옵션들 항목들을 콤마로 구분)
        9. parking (주차 가능 여부, True/False)
        10. check_in_time (체크인 시간,형식 : HH:MM)
        11. check_out_time (체크아웃 시간, 형식 : HH:MM)
        12. stay_type (숙소 형태, 예시:호텔, 모텔, 리조트, 풀빌라 등)
        13. foreign_yn (국내 숙소인지 외국 숙소인지 알려줘, 국내/외국)
        14. event (축제/콘서트 등 관련 이벤트가 포함되어있으면 축제,콘서트 이름을 써줘)
        """
    }

    query = {
        "role": "user",
        "content": f"""
        설명에서 필요한 14개의 필드를 추출하고 JSON 형식으로 반환해줘. 단, 필드는 14개 고정이야.
        
        설명: {description}
        """
    }

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 "gpt-4" 사용 가능
            messages=[prompt, query],
            temperature=0,
            max_tokens=1000
        )

        # 응답을 받아 JSON으로 변환
        result = response['choices'][0]['message']['content']
        result = result.strip()  # 공백 제거

        # json으로 시작되는 쓸데없는 부분을 제거
        if result.startswith("```json"):
            result = result.replace("```json", "").strip()
        if result.endswith("```"):
            result = result.replace("```", "").strip()

        # Pydantic을 사용해 응답을 구조화된 데이터로 변환
        parsed_data = Fields.parse_raw(result)
        return parsed_data.dict()  # 딕셔너리로 반환

    except Exception as e:
        print("응답 파싱 중 오류 발생:", e)
        return {field: np.nan for field in Fields.__fields__} # NaN 값으로 채운 딕셔너리 반환


In [18]:
# 멀티쓰레딩
def process_descriptions(descriptions):
    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(extract_fields, desc) for desc in descriptions]
        
        # tqdm으로 진행률 표시
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing descriptions"):
            results.append(future.result())
    
    return results

descriptions = df['description_1'].tolist()
processed_data = process_descriptions(descriptions)

Processing descriptions:   0%|          | 3/661 [00:02<06:14,  1.76it/s]

응답 파싱 중 오류 발생:응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
 You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/d

Processing descriptions:   1%|          | 7/661 [00:02<02:09,  5.04it/s]

응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


Processing descriptions:   2%|▏         | 13/661 [00:02<01:01, 10.51it/s]

...

응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/d

Processing descriptions:  99%|█████████▉| 657/661 [00:36<00:00, 20.30it/s]

응답 파싱 중 오류 발생:응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
 You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


Processing descriptions: 100%|██████████| 661/661 [00:36<00:00, 18.11it/s]

응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
응답 파싱 중 오류 발생: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.


In [10]:
processed_data

[{'expiration_stdate': nan,
  'expiration_endate': nan,
  'room_type': nan,
  'head_count': nan,
  'shipping_fee': nan,
  'transaction_method': nan,
  'market_price': nan,
  'options': nan,
  'parking': nan,
  'check_in_time': nan,
  'check_out_time': nan,
  'stay_type': nan,
  'foreign_yn': nan,
  'event': nan},
 {'expiration_stdate': nan,
  'expiration_endate': nan,
  'room_type': nan,
  'head_count': nan,
  'shipping_fee': nan,
  'transaction_method': nan,
  'market_price': nan,
  'options': nan,
  'parking': nan,
  'check_in_time': nan,
  'check_out_time': nan,
  'stay_type': nan,
  'foreign_yn': nan,
  'event': nan},
 {'expiration_stdate': nan,
  'expiration_endate': nan,
  'room_type': nan,
  'head_count': nan,
  'shipping_fee': nan,
  'transaction_method': nan,
  'market_price': nan,
  'options': nan,
  'parking': nan,
  'check_in_time': nan,
  'check_out_time': nan,
  'stay_type': nan,
  'foreign_yn': nan,
  'event': nan},
 {'expiration_stdate': nan,
  'expiration_endate': nan,

In [78]:
df1 = pd.DataFrame(processed_data)

In [84]:
df.shape

(491, 15)